<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

# Sprint Challenge
## *Data Science Unit 4 Sprint 1*

After a week of Natural Language Processing, you've learned some cool new stuff: how to process text, how turn text into vectors, and how to model topics from documents. Apply your newly acquired skills to one of the most famous NLP datasets out there: [Yelp](https://www.yelp.com/dataset/challenge). As part of the job selection process, some of my friends have been asked to create analysis of this dataset, so I want to empower you to have a head start.  

The real dataset is massive (almost 8 gigs uncompressed). I've sampled the data for you to something more managable for the Sprint Challenge. You can analyze the full dataset as a stretch goal or after the sprint challenge. As you work on the challenge, I suggest adding notes about your findings and things you want to analyze in the future.

## Challenge Objectives
*Successfully complete these all these objectives to earn a 2. There are more details on each objective further down in the notebook.*
* <a href="#p1">Part 1</a>: Write a function to tokenize the yelp reviews
* <a href="#p2">Part 2</a>: Create a vector representation of those tokens
* <a href="#p3">Part 3</a>: Use your tokens in a classification model on yelp rating
* <a href="#p4">Part 4</a>: Estimate & Interpret a topic model of the Yelp reviews

In [1]:
import pandas as pd

yelp = pd.read_json('./data/review_sample.json', lines=True)

In [2]:
yelp.head()

,business_id,cool,date,funny,review_id,stars,text,useful,user_id
0,nDuEqIyRc8YKS1q1fX0CZg,1,2015-03-31 16:50:30,0,eZs2tpEJtXPwawvHnHZIgQ,1,"BEWARE!!! FAKE, FAKE, FAKE....We also own a sm...",10,n1LM36qNg4rqGXIcvVXv8w
1,eMYeEapscbKNqUDCx705hg,0,2015-12-16 05:31:03,0,DoQDWJsNbU0KL1O29l_Xug,4,Came here for lunch Togo. Service was quick. S...,0,5CgjjDAic2-FAvCtiHpytA
2,6Q7-wkCPc1KF75jZLOTcMw,1,2010-06-20 19:14:48,1,DDOdGU7zh56yQHmUnL1idQ,3,I've been to Vegas dozens of times and had nev...,2,BdV-cf3LScmb8kZ7iiBcMA
3,k3zrItO4l9hwfLRwHBDc9w,3,2010-07-13 00:33:45,4,LfTMUWnfGFMOfOIyJcwLVA,1,We went here on a night where they closed off ...,5,cZZnBqh4gAEy4CdNvJailQ
4,6hpfRwGlOzbNv7k5eP9rsQ,1,2018-06-30 02:30:01,0,zJSUdI7bJ8PNJAg4lnl_Gg,4,"3.5 to 4 stars\n\nNot bad for the price, $12.9...",5,n9QO4ClYAS7h9fpQwa5bhA


## Part 1: Tokenize Function
<a id="#p1"></a>

Complete the function `tokenize`. Your function should
- accept one document at a time
- return a list of tokens

You are free to use any method you have learned this week.

In [3]:
import spacy

nlp = spacy.load('en_core_web_md')

In [4]:
def tokenize(doc):
    text = nlp(doc)
    tokens = str([token.lemma_ for token in text if (token.is_stop != True) and (token.is_punct != True)])
    return tokens

In [12]:
yelp['tokens'] = yelp['text'].apply(tokenize)

## Part 2: Vector Representation
<a id="#p2"></a>
1. Create a vector representation of the reviews
2. Write a fake review and query for the 10 most similiar reviews, print the text of the reviews. Do you notice any patterns?
    - Given the size of the dataset, it will probably be best to use a `NearestNeighbors` model for this. 

In [36]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.neighbors import NearestNeighbors

cv = CountVectorizer()
model = NearestNeighbors(n_neighbors=5, algorithm='ball_tree')

In [72]:
fake_review = "2 stars. A unique restraunt indeed. Serves food from menu different cuisines, each having its own menu. Different ways of serving the food each cuisine in order to savor the experience."

In [48]:
matrix = cv.fit_transform(list(yelp['text']))
dtm = pd.DataFrame(matrix.toarray())
dtm.head()

,0,1,2,3,4,5,6,7,8,9,...,27578,27579,27580,27581,27582,27583,27584,27585,27586,27587
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [57]:
import matplotlib.pyplot as plt

In [51]:
model.fit(dtm)

NearestNeighbors(algorithm='ball_tree', leaf_size=30, metric='minkowski',
         metric_params=None, n_jobs=None, n_neighbors=5, p=2, radius=1.0)

## Part 3: Classification
<a id="#p3"></a>
Your goal in this section will be to predict `stars` from the review dataset. 

1. Create a piepline object with a sklearn `CountVectorizer` or `TfidfVector` and any sklearn classifier. Use that pipeline to estimate a model to predict `stars`. Use the Pipeline to predict a star rating for your fake review from Part 2. 
2. Tune the entire pipeline with a GridSearch

In [59]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

In [87]:
cv2 = CountVectorizer()
rf = RandomForestClassifier()
pipeline = Pipeline([
    ('vect', cv2),
    ('clf', rf)
])
param_grid = {
    'vect__stop_words': [None],
    'vect__ngram_range': [(1,2), (1,3)],
    'vect__min_df': (0, 0.15),
    'vect__max_df': (0.55, 1.0),
}
gs = GridSearchCV(estimator=pipeline, param_grid=param_grid, n_jobs=-1, verbose=10, return_train_score=True)

In [88]:
gs.fit(yelp['text'], yelp['stars'])

Fitting 3 folds for each of 8 candidates, totalling 24 fits


/Users/ashwin/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:   32.5s
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:   48.1s
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:  1.4min
[Parallel(n_jobs=-1)]: Done  20 out of  24 | elapsed:  1.6min remaining:   19.2s
[Parallel(n_jobs=-1)]: Done  24 out of  24 | elapsed:  1.8min finished
/Users/ashwin/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


GridSearchCV(cv='warn', error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...obs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))]),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'vect__stop_words': [None], 'vect__ngram_range': [(1, 2), (1, 3)], 'vect__min_df': (0, 0.15), 'vect__max_df': (0.55, 1.0)},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=10)

In [89]:
gs.predict([fake_review])

array([5])

## Part 4: Topic Modeling

Let's find out what those yelp reviews are saying! :D

1. Estimate a LDA topic model of the review text
2. Create 1-2 visualizations of the results
    - You can use the most important 3 words of a topic in relevant visualizations. Refer to yesterday's notebook to extract. 
3. In markdown, write 1-2 paragraphs of analysis on the results of your topic model

__*Note*__: You can pass the DataFrame column of text reviews to gensim. You do not have to use a generator.

In [79]:
from gensim.corpora import Dictionary
from gensim.models.ldamulticore import LdaMulticore
from gensim.models import CoherenceModel

In [136]:
[list(yelp['tokens'])]

[["['beware', 'fake', 'fake', 'fake', 'small', 'business', 'Los', 'Alamitos', 'receive', 'look', 'like', 'legitimate', 'bill', '$', '70', 'account', 'number', ' ', 'call', 'phone', 'number', 'list', '866', '273', '7934', ' ', 'wait', 'time', 'hold', 'say', '20', 'minute', 'leave', 'message', ' ', 'live', 'person', 'phone', 'matter', 'number', 'select', ' ', 'leave', 'firm', 'message', 'contact', 'BBB', 'attorney', 'company', 'try', 'scam', 'business', 'illegal']",
  "['come', 'lunch', 'Togo', 'service', 'quick', 'Staff', 'friendly', 'complaint', 'sweet', 'tea', 'good', 'parking', 'pain']",
  "['Vegas', 'dozen', 'time', 'step', 'foot', 'Circus', 'Circus', ' ', 'reason', 'resort', 'market', 'famile', 'travel', 'Vegas', 'business', 'year', 'old', 'Circus', 'Circus', 'need', 'room', 'night', 'get', 'hotels.com', 'rate', '$', '24', 'pull', 'trigger', ' ', 'arrive', 'interior', 'nice', 'expect', 'check', 'go', 'fast', 'long', 'line', '\\n\\n', 'young', 'lady', 'check', 'explain', 'room', 'lo

In [137]:
dictionary = Dictionary([list(yelp['tokens'])])

In [138]:
corpus = [dictionary.doc2bow(token) for token in [yelp['tokens']]]

In [146]:
lda = LdaMulticore(corpus=corpus,
                   id2word=dictionary,
                   random_state=723812,
                   passes=10,
                   workers=4
)

In [147]:
lda.print_topics()

[(15,
  '0.000*"[\'love\', \'place\', \'Pho\', \'generally\', \'not\', \'order\', \'food\', \'average\', \'noodle\', \'soup\', \'Pho\', \'awesome\', \'\\n\\n\', \'advantage\', \'extra\', \'ingredient\', \'sauce\', \'soup\', \'snap\', \'timer\', \'think\', \'soup\', \'bland\']" + 0.000*"[\'ok\', \'give\', \'couple\', \'month\', \'return\', \'try\', \'definitely\', \'improve\', \'quality\', \'service\', \'staff\', \'frozen\', \'face\', \'see\', \'smile\', \'line\', \'move\', \'faster\', \'come\', \'regularly\', \'5\', \'star\', \'improve\', \'green\', \'tea\', \'drink\']" + 0.000*"[\'2\', \'kid\', \'attend\', \'Shenker\', \'Academy\', \'love\', \' \', \'daughter\', \'go\', \'previous\', \'school\', \'prior\', \'attend\', \'Shenker\', \'satisfy\', \'care\', \'receive\', \' \', \'change\', \'start\', \'Shenker\', \' \', \'administration\', \'teacher\', \'absolutely\', \'amazing\', \'kid\', \' \', \'lot\', \'fun\', \'school\', \'learn\', \' \', \'recommend\', \'Shenker\', \'parent\', \'look

In [148]:
distro = [lda[d] for d in corpus]
distro[0]

[]

In [149]:
lda[corpus[0]]

[]

In [151]:
df = pd.DataFrame.from_records(distro)
df.head()

""


## Stretch Goals

Complete one of more of these to push your score towards a three: 
* Incorporate named entity recognition into your analysis
* Compare vectorization methods in the classification section
* Analyze more (or all) of the yelp dataset - this one is v. hard. 
* Use a generator object on the reviews file - this would help you with the analyzing the whole dataset.
* Incorporate any of the other yelp dataset entities in your analysis (business, users, etc.)